# **Data Analysis using SQL**

In [27]:
# Importing the required libraries
import pandas as pd
import sqlite3
import json
import ast  # Library for literal_eval
from pytz import timezone
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Read the excel file into a pandas dataframe
organization=pd.read_excel('Data Analyst Intern SQL Assignment.xlsx',sheet_name='organization')
organization.head()

,org_id,org_name,org_code,org_status,org_date,properties
0,11,Company A,BIW,renewed,2023-05-05 04:23:04,"{""email"": ""biw@gmail.com"",""company"":""Company A..."
1,22,Company B,BIX,renewed,2023-05-12 06:28:34,"{""email"": ""bix@gmail.com"",""location"":""Punjab"",..."
2,33,Company C,BMR,not renewed,2023-06-15 06:53:38,"{""email"": ""bon@gmail.com"",""company"":""Company C..."
3,44,Company D,BMW,not renewed,2023-06-15 18:19:42,"{""email"": ""bou@gmail.com"",""company"":""Company D..."
4,55,Company E,BIZ,renewed,2023-05-15 09:55:47,"{""email"": ""biz@gmail.com"",""location"":""Gujarat""..."


In [10]:
# Preprocessing the data to make querying easier
# Convert 'org_date' column to datetime
organization['org_date'] = pd.to_datetime(organization['org_date'])

# Localize datetime to UTC time zone
organization['org_date'] = organization['org_date'].dt.tz_localize('UTC') #Universal co-ordinated time (to maintain consistency)

# Convert datetime to 'Asia/Kolkata' time zone
organization['org_date'] = organization['org_date'].dt.tz_convert('Asia/Kolkata')

In [11]:
# Convert 'properties' column from string to dictionary
organization['properties'] = organization['properties'].apply(ast.literal_eval)

# Extract 'location' from 'properties' column if it exists
organization['location'] = organization['properties'].apply(lambda x: x.get('location', None))
# Convert 'properties' column back to string

organization['properties'] = organization['properties'].apply(json.dumps)

In [12]:
organization.head()

,org_id,org_name,org_code,org_status,org_date,properties,location
0,11,Company A,BIW,renewed,2023-05-05 09:53:04+05:30,"{""email"": ""biw@gmail.com"", ""company"": ""Company...",Maharastra
1,22,Company B,BIX,renewed,2023-05-12 11:58:34+05:30,"{""email"": ""bix@gmail.com"", ""location"": ""Punjab...",Punjab
2,33,Company C,BMR,not renewed,2023-06-15 12:23:38+05:30,"{""email"": ""bon@gmail.com"", ""company"": ""Company...",Maharastra
3,44,Company D,BMW,not renewed,2023-06-15 23:49:42+05:30,"{""email"": ""bou@gmail.com"", ""company"": ""Company...",Gujarat
4,55,Company E,BIZ,renewed,2023-05-15 15:25:47+05:30,"{""email"": ""biz@gmail.com"", ""location"": ""Gujara...",Gujarat


In [13]:
call_log=pd.read_excel('Data Analyst Intern SQL Assignment.xlsx',sheet_name='call_log')
call_log.head()

,call_date,user_id,lead_id,org_id,call_connected,call_not_connected_reason
0,2023-05-23 05:37:14,f38197ca-8c60-467c-8c99-8f202b0aec14,add0dfb3-eff7-408c-b6e2-ab1f56dcb734,999,0,NOT_PICKED
1,2023-05-23 06:01:54,ae3fae95-5e3a-40cc-9c01-5779ec9d4010,ef76d2ca-c1a2-47ba-b48c-3ae18155cc88,999,1,NaN
2,2023-05-23 06:36:41,ae3fae95-5e3a-40cc-9c01-5779ec9d4010,334c9162-21ed-4bd3-bd02-d2c6a93d593b,999,0,NOT_PICKED
3,2023-05-23 07:32:40,ae3fae95-5e3a-40cc-9c01-5779ec9d4010,7948c411-dc04-4fac-ad49-dde5727c5109,999,0,other(Not interested)
4,2023-05-23 11:24:21,f38197ca-8c60-467c-8c99-8f202b0aec14,9376f9d1-bc61-4dbf-bb78-3bac588b0712,999,0,NOT_PICKED


In [14]:
# Connecting to sqlite database
conn=sqlite3.connect('Assignment.db')

In [15]:
# Converting organization dataframe to sql table
organization.to_sql('organization', conn, if_exists='replace', index=False)

20

In [16]:
# Converting call_log dataframe to sql table
call_log.to_sql('call_log',conn,if_exists='replace',index=False)

385

In [17]:
# Creating cursor object to execute SQL queries
cur=conn.cursor()

## 1. Find the first connected call for all the renewed organizations from the Gujarat location


In [87]:
# Execute the SQL query
cur.execute('''
    SELECT c.*
    FROM organization o
    JOIN call_log c ON o.org_id = c.org_id
    WHERE o.org_status = 'renewed'
    AND o.location = 'Gujarat'
    AND c.call_connected = 1
    ORDER BY c.call_date
    LIMIT 1
''')


In [88]:
# Fetch the column names
column_names = [description[0] for description in cur.description]
print("Column Names:", column_names)

# Fetch the result
result = cur.fetchone()
print("Fetched Result:", result)

# Convert the result to a DataFrame
result_df = pd.DataFrame([result], columns=column_names)
print("\nResult")
result_df

Column Names: ['call_date', 'user_id', 'lead_id', 'org_id', 'call_connected', 'call_not_connected_reason']
Fetched Result: (1684801914, 'ae3fae95-5e3a-40cc-9c01-5779ec9d4010', 'ef76d2ca-c1a2-47ba-b48c-3ae18155cc88', 999, 1, None)

Result


,call_date,user_id,lead_id,org_id,call_connected,call_not_connected_reason
0,1684801914,ae3fae95-5e3a-40cc-9c01-5779ec9d4010,ef76d2ca-c1a2-47ba-b48c-3ae18155cc88,999,1,None


In [91]:
cur.close()

## 2. Find the count of organizations that had three consecutive calls (excluding Saturday and Sunday) within 0-4 days, 5-8 days, 8-15 days, 16-30 days,30+ days of organization creation

#### For Non-Renewed organization

In [22]:
call_log['call_date'] = pd.to_datetime(call_log['call_date'])

In [23]:
merged_df = pd.merge(organization, call_log, on='org_id', how='inner')
merged_df.head()

,org_id,org_name,org_code,org_status,org_date,properties,location,call_date,user_id,lead_id,call_connected,call_not_connected_reason
0,11,Company A,BIW,renewed,2023-05-05 09:53:04+05:30,"{""email"": ""biw@gmail.com"", ""company"": ""Company...",Maharastra,2023-05-29 06:55:55,b2b4a5a9-df17-453b-b4ce-e69deea22ee4,5797ed22-fe18-48b7-ad20-e35d8440af52,0,NOT_PICKED
1,11,Company A,BIW,renewed,2023-05-05 09:53:04+05:30,"{""email"": ""biw@gmail.com"", ""company"": ""Company...",Maharastra,2023-06-20 10:34:12,b2b4a5a9-df17-453b-b4ce-e69deea22ee4,84faad73-e58f-4396-bc0e-af4c70987d71,1,NaN
2,11,Company A,BIW,renewed,2023-05-05 09:53:04+05:30,"{""email"": ""biw@gmail.com"", ""company"": ""Company...",Maharastra,2023-06-20 12:10:08,b2b4a5a9-df17-453b-b4ce-e69deea22ee4,f7cdbcdf-a073-4f0d-b38e-b0b76ee7bef4,1,NaN
3,11,Company A,BIW,renewed,2023-05-05 09:53:04+05:30,"{""email"": ""biw@gmail.com"", ""company"": ""Company...",Maharastra,2023-06-21 05:21:50,b2b4a5a9-df17-453b-b4ce-e69deea22ee4,2f0916c4-b63d-4120-aba9-ab0fbfa6ef66,1,NaN
4,11,Company A,BIW,renewed,2023-05-05 09:53:04+05:30,"{""email"": ""biw@gmail.com"", ""company"": ""Company...",Maharastra,2023-06-21 06:35:19,b2b4a5a9-df17-453b-b4ce-e69deea22ee4,1fd028b9-e17c-4fe9-af75-b660aeb84a7e,1,NaN


In [24]:
# Filter out non-renewed organizations
non_renewed_df = merged_df[merged_df['org_status'] == 'not renewed']

In [28]:
# Make datetime columns timezone-naive
non_renewed_df['org_date'] = non_renewed_df['org_date'].dt.tz_localize(None)
non_renewed_df['call_date'] = non_renewed_df['call_date'].dt.tz_localize(None)

# Calculate time difference excluding Saturdays and Sundays
non_renewed_df['days_since_creation'] = (non_renewed_df['call_date'] - non_renewed_df['org_date']).dt.days

# Exclude Saturdays and Sundays
non_renewed_df['weekday'] = non_renewed_df['org_date'].dt.dayofweek
non_renewed_df = non_renewed_df[non_renewed_df['weekday'] < 5]

In [29]:
# Define bins
bins = [-1, 4, 8, 15, 30, float('inf')]
labels = ['0-4 days', '5-8 days', '8-15 days', '16-30 days', '30+ days']
# Bin the data
non_renewed_df['bin'] = pd.cut(non_renewed_df['days_since_creation'], bins=bins, labels=labels)
# Count organizations falling into each bin
counts = non_renewed_df.groupby('bin').size()
# Display the counts
print("Count of Org - Not Renewed")
for label, count in counts.items():
    print(f"{label}: {count}")

Count of Org - Not Renewed
0-4 days: 34
5-8 days: 51
8-15 days: 2
16-30 days: 1
30+ days: 66


#### For renewed organization

In [30]:
# Filter out renewed organizations
renewed_df = merged_df[merged_df['org_status'] == 'renewed']
# Make datetime columns timezone-naive
renewed_df['org_date'] = renewed_df['org_date'].dt.tz_localize(None)
renewed_df['call_date'] = renewed_df['call_date'].dt.tz_localize(None)

# Calculate time difference excluding Saturdays and Sundays
renewed_df['days_since_creation'] = (renewed_df['call_date'] - renewed_df['org_date']).dt.days

# Exclude Saturdays and Sundays
renewed_df['weekday'] = renewed_df['org_date'].dt.dayofweek
renewed_df = renewed_df[renewed_df['weekday'] < 5]
# Define bins
bins = [-1, 4, 8, 15, 30, float('inf')]
labels = ['0-4 days', '5-8 days', '8-15 days', '16-30 days', '30+ days']
# Bin the data
renewed_df['bin'] = pd.cut(renewed_df['days_since_creation'], bins=bins, labels=labels)
# Count organizations falling into each bin
counts = renewed_df.groupby('bin').size()
# Display the counts
print("Count of Org - Not Renewed")
for label, count in counts.items():
    print(f"{label}: {count}")

Count of Org - Not Renewed
0-4 days: 13
5-8 days: 27
8-15 days: 12
16-30 days: 21
30+ days: 137


## 3. Identify the location with the maximum number of connected calls for unique leads

In [34]:
cur=conn.cursor() #Creating cursor object to execute the query

In [51]:
cur.execute('''
SELECT org.location, COUNT(DISTINCT call_log.lead_id) AS unique_leads_count
FROM organization AS org
JOIN call_log ON org.org_id = call_log.org_id
WHERE call_log.call_connected = 1
GROUP BY org.location
ORDER BY unique_leads_count DESC
LIMIT 1;
''')

In [52]:
# Fetch the column names
column_names = [description[0] for description in cur.description]
print("Column Names:", column_names)

# Fetch the result
result = cur.fetchall()

# Convert the result to a DataFrame
result_df = pd.DataFrame(result, columns=column_names)
print("\nResult")
result_df

Column Names: ['location', 'unique_leads_count']

Result


,location,unique_leads_count
0,Maharastra,76


In [39]:
cur.close()

## 4. For calls not connected, identify the most common reason(s) for why the call was not connected.

In [40]:
cur=conn.cursor()

In [49]:
cur.execute('''
SELECT call_not_connected_reason, COUNT(*) AS reason_count
FROM call_log
WHERE call_connected = 0
GROUP BY call_not_connected_reason
ORDER BY reason_count DESC;
''')

In [50]:
# Fetch the column names
column_names = [description[0] for description in cur.description]
print("Column Names:", column_names)

# Fetch the result
result = cur.fetchall()

# Convert the result to a DataFrame
result_df = pd.DataFrame(result, columns=column_names)
print("\nResult")
result_df

Column Names: ['call_not_connected_reason', 'reason_count']

Result


,call_not_connected_reason,reason_count
0,NOT_PICKED,107
1,BUSY,16
2,INVALID_NUMBER,15
3,SWITCH_OFF,14
4,USER_DISCONNECTED,13
5,CALL_NOT_CONNECTED,11
6,NUMBER_NOT_IN_USE,8
7,NETWORK_ISSUE,6
8,INCOMING_CALLS_NOT_AVAILABLE,2
9,other(Training),1


### **The most common reason for the call not being connected is that it was not answered.**

## Summary

#### First Connected Call for Renewed Organizations from Gujarat Location:
    The first connected call for a renewed organization from the Gujarat location occurred on the date 1684801914.

#### Count of Organizations with Three Consecutive Calls:
- For renewed organizations:
    - 0-4 days: 34 organizations
    - 5-8 days: 51 organizations
    - 8-15 days: 2 organizations
    - 16-30 days: 1 organization
    - 30+ days: 66 organizations
- For not renewed organizations:
    - 0-4 days: 13 organizations
    - 5-8 days: 27 organizations
    - 8-15 days: 12 organizations
    - 16-30 days: 21 organizations
    - 30+ days: 137 organizations
    
#### Location with Maximum Number of Connected Calls for Unique Leads:
    The location with the maximum number of connected calls for unique leads is Maharastra, with a count of 76 unique leads.

#### Most Common Reasons for Calls Not Connected:
    The most common reason for calls not connected is "NOT_PICKED," with a count of 107 occurrences. Other common reasons include "BUSY," "INVALID_NUMBER," "SWITCH_OFF," and "USER_DISCONNECTED."


## Inference:

- The analysis provides insights into the call connectivity status, timing of consecutive calls, and common reasons for calls not being connected.
- Renewed organizations in Gujarat show a significant number of consecutive calls within different time frames, indicating active engagement.
- The analysis highlights the importance of follow-up strategies, especially for not renewed organizations, where the distribution of consecutive calls varies across different time intervals.
- Maharastra emerges as a location with a high number of connected calls for unique leads, suggesting potential opportunities for business engagement and growth.
- The most common reason for calls not being connected is related to the call recipient not picking up the phone, indicating a need for improved call response strategies or alternative communication methods.